# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 24 2022 2:02PM,249274,20,ATR-I000014,"HVL, LLC dba Atrium Innovations",Released
1,Oct 24 2022 1:57PM,249273,10,8688518,"ClearSpec, LLC",Released
2,Oct 24 2022 1:54PM,249272,10,CLI00406.1,"CLINUVEL, Inc.",Released
3,Oct 24 2022 1:54PM,249272,10,CLI00407.1,"CLINUVEL, Inc.",Released
4,Oct 24 2022 1:54PM,249272,10,CLI00408.1,"CLINUVEL, Inc.",Released
5,Oct 24 2022 1:47PM,249271,10,MSP215160,"Methapharm, Inc.",Released
6,Oct 24 2022 1:47PM,249271,10,MSP215161,"Methapharm, Inc.",Released
7,Oct 24 2022 1:47PM,249271,10,MSP215162,"Methapharm, Inc.",Released
8,Oct 24 2022 1:47PM,249271,10,MSP215163,"Methapharm, Inc.",Released
9,Oct 24 2022 1:47PM,249271,10,MSP215164,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,249270,Released,1
19,249271,Released,13
20,249272,Released,3
21,249273,Released,1
22,249274,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249270,NaN,1.0
249271,NaN,13.0
249272,NaN,3.0
249273,NaN,1.0
249274,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249218,0.0,1.0
249229,0.0,59.0
249232,0.0,1.0
249236,0.0,2.0
249241,25.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249218,0,1
249229,0,59
249232,0,1
249236,0,2
249241,25,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249218,0,1
1,249229,0,59
2,249232,0,1
3,249236,0,2
4,249241,25,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249218,,1
1,249229,,59
2,249232,,1
3,249236,,2
4,249241,25,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 24 2022 2:02PM,249274,20,"HVL, LLC dba Atrium Innovations"
1,Oct 24 2022 1:57PM,249273,10,"ClearSpec, LLC"
2,Oct 24 2022 1:54PM,249272,10,"CLINUVEL, Inc."
5,Oct 24 2022 1:47PM,249271,10,"Methapharm, Inc."
18,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation
31,Oct 24 2022 1:46PM,249267,10,ISDIN Corporation
61,Oct 24 2022 1:42PM,249270,19,"GCH Granules USA, Inc."
62,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc."
83,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix
84,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 24 2022 2:02PM,249274,20,"HVL, LLC dba Atrium Innovations",,1
1,Oct 24 2022 1:57PM,249273,10,"ClearSpec, LLC",,1
2,Oct 24 2022 1:54PM,249272,10,"CLINUVEL, Inc.",,3
3,Oct 24 2022 1:47PM,249271,10,"Methapharm, Inc.",,13
4,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation,,13
5,Oct 24 2022 1:46PM,249267,10,ISDIN Corporation,,30
6,Oct 24 2022 1:42PM,249270,19,"GCH Granules USA, Inc.",,1
7,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc.",13,8
8,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix,,1
9,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc.",,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 2:02PM,249274,20,"HVL, LLC dba Atrium Innovations",1,
1,Oct 24 2022 1:57PM,249273,10,"ClearSpec, LLC",1,
2,Oct 24 2022 1:54PM,249272,10,"CLINUVEL, Inc.",3,
3,Oct 24 2022 1:47PM,249271,10,"Methapharm, Inc.",13,
4,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation,13,
5,Oct 24 2022 1:46PM,249267,10,ISDIN Corporation,30,
6,Oct 24 2022 1:42PM,249270,19,"GCH Granules USA, Inc.",1,
7,Oct 24 2022 1:18PM,249265,20,"Exact-Rx, Inc.",8,13
8,Oct 24 2022 12:58PM,249264,10,Yusen – 3D Matrix,1,
9,Oct 24 2022 12:51PM,249263,10,"Methapharm, Inc.",6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 2:02PM,249274,20,"HVL, LLC dba Atrium Innovations",1,
1,Oct 24 2022 1:57PM,249273,10,"ClearSpec, LLC",1,
2,Oct 24 2022 1:54PM,249272,10,"CLINUVEL, Inc.",3,
3,Oct 24 2022 1:47PM,249271,10,"Methapharm, Inc.",13,
4,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation,13,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 2:02PM,249274,20,"HVL, LLC dba Atrium Innovations",1,NaN
1,Oct 24 2022 1:57PM,249273,10,"ClearSpec, LLC",1,NaN
2,Oct 24 2022 1:54PM,249272,10,"CLINUVEL, Inc.",3,NaN
3,Oct 24 2022 1:47PM,249271,10,"Methapharm, Inc.",13,NaN
4,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation,13,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 24 2022 2:02PM,249274,20,"HVL, LLC dba Atrium Innovations",1,0.0
1,Oct 24 2022 1:57PM,249273,10,"ClearSpec, LLC",1,0.0
2,Oct 24 2022 1:54PM,249272,10,"CLINUVEL, Inc.",3,0.0
3,Oct 24 2022 1:47PM,249271,10,"Methapharm, Inc.",13,0.0
4,Oct 24 2022 1:46PM,249269,10,ISDIN Corporation,13,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3240346,139,3.0
15,249241,2,25.0
18,249252,1,0.0
19,997011,14,0.0
20,498539,9,13.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3240346,139,3.0
1,15,249241,2,25.0
2,18,249252,1,0.0
3,19,997011,14,0.0
4,20,498539,9,13.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,139,3.0
1,15,2,25.0
2,18,1,0.0
3,19,14,0.0
4,20,9,13.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,139.0
1,15,Released,2.0
2,18,Released,1.0
3,19,Released,14.0
4,20,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20
Status,,,,,
Executing,3.0,25.0,0.0,0.0,13.0
Released,139.0,2.0,1.0,14.0,9.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20
0,Executing,3.0,25.0,0.0,0.0,13.0
1,Released,139.0,2.0,1.0,14.0,9.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20
0,Executing,3.0,25.0,0.0,0.0,13.0
1,Released,139.0,2.0,1.0,14.0,9.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()